In [70]:
# ! CURL_CA_BUNDLE=/ca-certificates.crt pip install polars scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 292.3 kB/s eta 0:00:0000:0100:04


In [7]:
from itertools import product
import polars as pl
import pandas as pd

In [ ]:
tissues = ["BAT", "BLOOD", "LIVER", "HEART", "WAT-SC", "SKM-GN", "HYPOTH"]
assays = ["ATAC", "TRNSCRPT", "METHYL"]

products = list(product(tissues, assays))

In [ ]:
genomic_data = (
    pl.read_csv("data/TRAINING_REGULATED_NORM_DATA.csv", null_values=["NA"])
    .filter(pl.col("assay").is_in(assays))
    .drop("dataset")
)
feature_to_gene = pl.read_csv("data/FEATURE_TO_GENE.csv", null_values=["NA"])

In [75]:
data: pl.DataFrame = genomic_data.join(
    feature_to_gene.select(pl.col("feature_ID", "ensembl_gene")),
    on="feature_ID",
    how="left",
).drop(pl.selectors.by_index(0))

In [96]:
grouped_data = (
    data.drop("tissue", "assay", "feature_ID")
    .group_by("feature", "ensembl_gene")
    .mean()
    .sort("ensembl_gene")
    .drop("ensembl_gene")
)

grouped_data.write_parquet("data/grouped_data.parquet")

In [ ]:
grouped_data = pl.read_parquet("data/grouped_data.parquet")

In [95]:
# Reading phenotype data
pheno = pd.read_csv("data/motrpac_pass1b-06_pheno_viallabel_data_merged_v4.0.txt")

# Sub-setting phenotype data to include necessary columns
phenoSubset = pheno[
    [
        "pid",
        "viallabel",
        "sex",
        "vo2.max.test.vo2_max_2",
        "registration.weight",
        "terminal.weight.bw",
        "group",
    ]
]

EmptyDataError: No columns to parse from file

In [94]:

phenoSubset["mass_lost"] = (
    phenoSubset["registration.weight"] - phenoSubset["terminal.weight.bw"]
)

phenoSubset["norm_mass_lost"] = (
    phenoSubset["mass_lost"] / phenoSubset["registration.weight"]
)

phenoSubset["norm_mass_lost"] = (
    phenoSubset["mass_lost"] / phenoSubset["registration.weight"]
)

EmptyDataError: No columns to parse from file

In [93]:
feature_engineered_data = grouped_data.select(pl.exclude("feature")).transpose(include_header=True, header_name="pid", column_names=grouped_data.get_column("feature"))